In [6]:
import os
import cv2
from keras.utils import Sequence
from keras.utils import np_utils




class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args:
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """

    def __init__(self, directory, batch_size=1, shuffle=True, data_augmentation=True):
        # Initialize the params
        self.batch_size = batch_size
        self.directory = directory
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data()
        # Print basic statistics information
        self.print_stats()


    def search_data(self):
        X_path = []
        Y_dict = {}
        # list all kinds of sub-folders
        self.dirs = sorted(os.listdir(self.directory))
        one_hots = np_utils.to_categorical(range(len(self.dirs)))
        for i, folder in enumerate(self.dirs):
            folder_path = os.path.join(self.directory, folder)
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                # append the each file path, and keep its label
                X_path.append(file_path)
                Y_dict[file_path] = one_hots[i]
        return X_path, Y_dict

    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = len(self.dirs)
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files, self.n_classes))
        for i, label in enumerate(self.dirs):
            print('%10s : ' % (label), i)
        return None

    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y

    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def read_image(self,img_path, H, W):
        #print(f"in read img\n{img_path}\n")
        # img_path=img_path+".jpg"
        #print(f"in read img--{img_path}")
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (W, H))  # you can resize to  (128,128) or (256,256)

        return img

    def data_generation(self, batch_path):
        #print(f"in data generation\n {batch_path}\n")
        # load data into memory, you can change the np.load to any method you want
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y

    def load_data(self, path):
        # load the processed .npy files which have 5 channels (1-3 for RGB, 4-5 for optical flows)
        #data = np.load(path, mmap_mode='r')
        #print(f"[+][+] in load data:path-->{path}]n")
        data = self.read_image(path, 224, 224)
        data = np.float32(data)
        return data


In [8]:
from keras.models import model_from_json
import gc
import cv2
import numpy as np
from keras.optimizers import adam, Adam
from sklearn import metrics



def read_image(img_path,H=224,W=224):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W,H)) # you can resize to  (128,128) or (256,256)
    return img

def load_model():
    # load json and create model
    json_file = open(r"E:\FINAL_PROJECT_DATA\FashioNet\Models\model.json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model_final = model_from_json(loaded_model_json)
    # load weights into new model
    model_final.load_weights(r"E:\FINAL_PROJECT_DATA\FashioNet\Models\save_weights.h5")
    adam = Adam(lr=1e-3)
    model_final.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    print("Loaded model from disk")
    return model_final

def get_y_true_label(y_true_vec):
    pre_dict = {0: "Blouses", 1: "Dresses", 2: "Shorts", 3: "Skirts"}
    pre_dict2 = {"Blouses":'B',"Dresses":'D' ,"Shorts":'SH',"Skirts":'SK'}
    # Get the maximum element from a Numpy array
    maxElement = np.amax(y_true_vec)
    #print('Max element from Numpy Array : ', maxElement)
    # Get the indices of maximum element in numpy array
    result = np.where(y_true_vec == np.amax(y_true_vec))
    #print('Returned tuple of arrays :', result)
    #print('List of Indices of maximum element :', result[0])
    ##print(f"[+][+] label = {pre_dict.get(int(result[0]))}\n")
    key = result[0][0]

    #LABEL = pre_dict.get(int(result[0]))
    LABEL = pre_dict.get(int(key))
    ##print(f"1[+][+][+] label = {pre_dict2.get(LABEL)}\n")
    return pre_dict2.get(LABEL)

def prediction_processes(pred):
    pred = pred[0]
    pre_dict={0:"Blouses",1:"Dresses",2:"Shorts",3:"Skirts"}
    pre_dict2 = {"Blouses": 'B', "Dresses": 'D', "Shorts": 'SH', "Skirts": 'SK'}
    categorical = """
                Blouses_Shirts :  0
                Dresses :  1
                Shorts :  2
                Skirts :  3
            """

    #print(f"pred len={len(pred)}\n")
    #for i in range(len(pred)):
        # print(f"pred{i}={round(pred[i], 5)}\n")
        #print(f"pred{i}={pred[i]}\n")

    # Get the maximum element from a Numpy array
    maxElement = np.amax(pred)
    #print('Max element from Numpy Array : ', maxElement)
    ## Get the indices of maximum element in numpy array
    result = np.where(pred == np.amax(pred))
    #print('Returned tuple of arrays :', result)
    #print('List of Indices of maximum element :', result[0])
    key = result[0][0]
    #print(f"Res = {pre_dict.get(key)}")
    #LABEL = pre_dict.get(int(result[0]))
    LABEL = pre_dict.get(int(key))
    #print(f"2[+][+][+] label = {pre_dict2.get(LABEL)}\n")
    return pre_dict2.get(LABEL)



def test_sklearn_metrics(y_true,y_pred):

    #
    # # Constants
    # C = "Cat"
    # F = "Fish"
    # H = "Hen"
    #print(y_true)
    #print(y_pred)
    # True values
    #y_true = [C, C, C, C, C, C, F, F, F, F, F, F, F, F, F, F, H, H, H, H, H, H, H, H, H]
    # Predicted values
    #y_pred = [C, C, C, C, H, F, C, C, C, C, C, C, H, H, F, F, C, C, C, H, H, H, H, H, H]
    # Print the confusion matrix
    print(metrics.confusion_matrix(y_true, y_pred))
    # Print the precision and recall, among other metrics
    print(metrics.classification_report(y_true, y_pred, digits=3))



model_final = load_model()

directory =r"F:\data for recommender system\FINAL_DATA_SORT\Test"
X= DataGenerator(directory,batch_size=2, shuffle=True, data_augmentation=False)

###############################################3
start = 0
end = 1
sample = len(X.X_path)
print("Number of sample={}\n".format(sample))

y_true = []
y_predictions_list = []

while end <= len(X.X_path):
    path = X.X_path[start:end]
    start = end
    end = end + 1
    batch_x, batch_y = X.data_generation(path)
    batch_y = batch_y[0]
    true_label_index = get_y_true_label(batch_y)

    y_true.append(true_label_index)

    #print(batch_x)
    pred = model_final.predict(batch_x, verbose=0)
    y_pred = prediction_processes(pred)

    y_predictions_list.append(y_pred)

test_sklearn_metrics(y_true,y_predictions_list)



Loaded model from disk
Found 6666 files belonging to 4 classes.
Blouses_Shirts :  0
   Dresses :  1
    Shorts :  2
    Skirts :  3
Number of sample=6666

[[2171  347  142  119]
 [ 251 1690   25  215]
 [  95   67  700  207]
 [  20   79   31  507]]
              precision    recall  f1-score   support

           B      0.856     0.781     0.817      2779
           D      0.774     0.775     0.775      2181
          SH      0.780     0.655     0.712      1069
          SK      0.484     0.796     0.602       637

    accuracy                          0.760      6666
   macro avg      0.723     0.752     0.726      6666
weighted avg      0.781     0.760     0.766      6666

